In [1]:
include("src/model.jl")

Main.CESModel

## Loading in Data

In [2]:
data = CESModel.read_data("I-O_DE2019_formatiert.csv")

## Setting Elasticities

Es können verschiedene Elastizitäten gesetzt werden, die hier verwendeten sind übernommen aus B&F 2019 
(el. of substitiution of inermediates = 0.0001, el labor/intermediates  = 0.5 el. of consumption = 0.9)

In [ ]:
elasticities = CESModel.Elasticities(0.0001, 0.5, 0.9)

## Setting Shocks

Shocks können entweder mit `calculate_investment!` in  1000€ eingegeben werden, oder die einzelnen Elemente im Vektor können auch direkt verändernt werden (z.b `supply_shock[12] = 1.2`), was dann einem Anstieg an demand von 20% entsrpicht.

In [ ]:
deman_shock = ones(71)
supply_shock = ones(71)
shocks = CESModel.Shocks(supply_shock,deman_shock) 
#shocks.demand_shock[35] = 1.09
CESModel.calculate_investment!(shocks,data,25000,"Vorb.Baustellen-,Bauinstallations-,Ausbauarbeiten")

## Full labor reallocation

In [ ]:
price,quantity = CESModel.solve_ces_model(data,shocks,elasticities)

In [ ]:
println("Real GDP: $(CESModel.real_gdp(price,quantity,data))")
println("Nominal GDP: $(CESModel.nominal_gdp(price,quantity,data))")


## No labor reallocation 

Es können verschiedene Funktionen angegeben werden,die zurückgeben wie viel Arbeit in den einzelnen Sektoren verfügbar. Hier in diesem Fall, ist keine zusätzliche Arbeit verfügbar, die funktion könnte aber auch zum Beispiel so aussehen: 
```julia
labor_realloc(data) = min(1.05 * data.labor_share,inv(I - diagm(1 .- data.factor_share) * data.Ω) * (data.consumption_share_gross_output .* ((data.shocks.demand_shock .* data.labor_share) - data.labor_share)) + data.labor_share)
``` 
(maximal 5% der verfügbaren Arbeit kann zusätzlich verwendet werden um der gesteigerten Nachfrage nachzukommen)

In [ ]:
labor_realloc(data) = data.labor_share #no labor_reallocation

In [ ]:
price,quantity = CESModel.solve_ces_model(data,shocks,elasticities,labor_reallocation = labor_realloc)

In [ ]:
println("Real GDP: $(CESModel.real_gdp(price,quantity,data))")
println("Nominal GDP: $(CESModel.nominal_gdp(price,quantity,data))")

In [ ]:
using LinearAlgebra
using Distributed, SharedArrays
shock_values = LinRange(1.0,1.8,50)
supply_shock = ones(71)

GDP_no_realloc_nominal = SharedArray(zeros(71,51))
GDP_nominal = SharedArray(zeros(71,50))

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init,labor_reallocation = labor_realloc)
        GDP_no_realloc_nominal[i,k] = CESModel.nominal_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init)
        GDP_nominal[i,k] = CESModel.nominal_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end

In [ ]:
findmax(mapslices(row -> maximum(collect(LinRange(1,maximum(row),50)) - row),GDP,dims=2))

In [ ]:
using DataFrames, CSV


In [ ]:
using Plots
scatter()
for row in eachrow(GDP_no_realloc)
    scatter!(row)
end
scatter!()

In [ ]:
scatter()
for row in eachrow(GDP)
    scatter!(row)
end
scatter!()

In [ ]:
plot()
for row in eachrow(GDP)
    plot!(row -  LinRange(1,maximum(row),50))
end
plot!()

In [ ]:
shock_values = LinRange(1.0,1.8,50)
supply_shock = ones(71)

GDP_no_realloc = SharedArray(zeros(71,51))
GDP = SharedArray(zeros(71,50))

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init,labor_reallocation = labor_realloc)
        GDP_no_realloc[i,k] = CESModel.real_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init)
        GDP[i,k] = CESModel.real_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end


In [ ]:
function savetabular(filname,gdp)
    sectors = data.io.Sektoren[1:71]
    df = DataFrame(gdp[1:end,:]',sectors)
    df[:,:DemandShockAmount] = collect(shock_values)
    CSV.write(joinpath(pwd(),"data",filname),df)
end